# Scratchwork: Cayley-Dickson Algebras

The purpose of this notebook is to develop the functions required to perform the Cayley-Dickson construction within the ``finite_algebras`` module.

In [1]:
import finite_algebras as alg
import numpy as np

## The Cayley-Dickson Construction

The [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) begins with a <i>base</i> algebra, $A$, usually a ring or field, from which another algebra, $\mathscr{C}(A)$, is derived, with compound elements, identical to those found in the direct product of $A$ with itself, and, like direct products, addition is defined element-wise, but multiplication is defined similar to complex number multiplication, as described in definition 1, below.

> **Definition 1: Cayley-Dickson Algebra (CDA)**
> 
> Let $A = \langle S, +, \cdot \rangle$ be a ring and let $T = S \times S$ (i.e., the cross-product of $S$ with itself).
> 
> Then, define the algebra, $\mathscr{C}(A) \equiv \langle T, \oplus, \odot \rangle$,
> 
> where $\forall (a,b),(c,d) \in T$,
> 
> $(a,b) \oplus (c,d) \equiv (a + c, b + d)$
> 
> $(a,b) \odot (c,d) \equiv (a \cdot c - b \cdot d, a \cdot d + b \cdot c)$

So, for example, if we apply the Cayley-Dickson construction to the field of real numbers, $\mathbb{R}$, we obtain the algebra of **complex numbers**, $\mathscr{C}(\mathbb{R}) \equiv \mathbb{C}$.
 
In fact, the Cayley-Dickson Algebra (CDA) constructor, $\mathscr{C}$, can be applied to multiple times, as described in definition 2, below.

> **Definition 2: CDA Constructor**
> 
> Let $k \in Z^+$ and $\mathscr{C}^0(A) \equiv A$,
> 
> then $\mathscr{C}^k(A) \equiv \mathscr{C}(\mathscr{C}^{k-1}(A))$,

So, for example, $\mathscr{C}^2(\mathbb{R}) \equiv \mathscr{C}(\mathbb{C}) \equiv \mathbb{H}$ is the algebra of **quaternions**, and $\mathscr{C}^3(\mathbb{R}) \equiv \mathbb{O}$ is the algebra of **octonions**.

In many (all?) papers on the subject, a generalization of complex multiplication is used, instead of the multiplication described above. And, there are multiple, different versions of the generalization.

All of the generalizations begin with a common, recursive definition of <i>conjugation</i>, described in definition 3, below.

> **Definition 3: Conjugation**
> 
> The **conjugate** of an algebraic element, $r$, is denoted by $\overline{r}$, and defined recursively as follows:
> 
> For $a \in A$, the base algebra, let $\overline{a} \equiv a$.
> 
> Then, let $k \in Z^+$; $p,q \in \mathscr{C}^{k-1}(A)$; and $(p,q) \in \mathscr{C}^k(A)$,
> 
> and define $\overline{(p, q)} \equiv (\overline{p}, -q)$

In **Shafer (1966)**, the generalization of CDA multiplication is defined as shown in definition 4, below.

> **Definition 4: CDA Multiplication in [Schafer 1966]**
> 
> Let $k \in Z^+$; $a,b,c,d,\mu \in \mathscr{C}^{k-1}(A)$; and $\mu \neq 0$,
> 
> then define multiplication for $(a,b), (c,d) \in \mathscr{C}^k(A)$ as
>
> $(a, b) \odot (c, d) = (a \cdot c + \mu \cdot d \cdot \overline{b}, \overline{a} \cdot d + c \cdot b)$

In **Shafer (1953)**, a different generalization of multiplication is given. See definition 5, below.

> **Definition 5: CDA Multiplication in [Schafer 1953]**
> 
> $(a, b) \odot (c, d) = (a \cdot c + \mu \cdot \overline{d} \cdot b, d \cdot a + b \cdot \overline{c})$

## Bibliography

* Schafer, R. D., "An Introduction to Nonassociative Algebras", Academic Press, 1966. ([or see this 1961 version](https://www.gutenberg.org/ebooks/25156))
* [Python code for octonion and sedenion multiplication](https://www.johndcook.com/blog/2018/07/09/octonioin-multiplication/) - John D Cook blog
* [The Octonions](https://web.archive.org/web/20180216125124/http://math.ucr.edu:80/home/baez/octonions/) - The Wayback Machine
* [Algebra over a field](https://en.wikipedia.org/wiki/Algebra_over_a_field) - Wikipedia
* [Cayley-Dickson construction](https://en.wikipedia.org/wiki/Cayley%E2%80%93Dickson_construction) - Wikipedia
* [Cayley-Dickson algebra](https://encyclopediaofmath.org/wiki/Cayley-Dickson_algebra) - Encyclopedia of Math
* [What comes after the ducentiquinquagintasexions?](https://english.stackexchange.com/questions/234607/what-comes-after-the-ducentiquinquagintasexions) - StackExchange
* ["Equivalence in a Class of Division Algebras of Order 16"](https://core.ac.uk/reader/82141950) by R. D. Schafer

## Algebras

### Algebras by "Squaring"

In [2]:
F3 = alg.generate_algebra_mod_n(3)
F3sqr = F3.make_cayley_dickson_algebra()

R4 = alg.generate_algebra_mod_n(4)  # Will be a ring because 4 is not prime
R4sqr = R4.make_cayley_dickson_algebra()

F5 = alg.generate_algebra_mod_n(5)  # Will also be a ring because 5 is not a Gaussian prime
F5sqr = F5.make_cayley_dickson_algebra()

F7 = alg.generate_algebra_mod_n(7)
F7sqr = F7.make_cayley_dickson_algebra()

In [3]:
F3.about()


** Field **
Name: F3
Instance ID: 4421828368
Description: Autogenerated Field of integers mod 3
Order: 3
Identity: '0'
Commutative? Yes
Cyclic?: Yes
  Generators: ['1', '2']
Elements:
   Index   Name   Inverse  Order
      0     '0'     '0'       1
      1     '1'     '2'       3
      2     '2'     '1'       3
Cayley Table (showing indices):
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]
Mult. Identity: '1'
Mult. Commutative? Yes
Zero Divisors: None
Multiplicative Cayley Table (showing indices):
[[0, 0, 0], [0, 1, 2], [0, 2, 1]]


In [4]:
print(f"{F3sqr.name} is a {F3sqr.__class__.__name__}")
print(f"{R4sqr.name} is a {R4sqr.__class__.__name__}")
print(f"{F5sqr.name} is a {F5sqr.__class__.__name__}")
print(f"{F7sqr.name} is a {F7sqr.__class__.__name__}")

F3_Sqr is a Field
R4_Sqr is a Ring
F5_Sqr is a Ring
F7_Sqr is a Field


### One Application of Cayley-Dickson

Here, one application of Cayley-Dickson construction process is applied to the F3 & F5 fields.

Note:

* 3 - Gaussian prime
* 4 - Not a prime
* 5 - Prime, but not a Gaussian prime

**Conclusion**: F3sqr, F3cda, F3cda66, & F3cda53 are all identical. Similar results for F4 & F7, and almost for F5, except that F5sqr and F5cda53 are different. (Why?)

In [5]:
F3cda = F3.make_cayley_dickson_algebra()
print(f"{F3cda.description}")
print(f"F3sqr == F3cda ?: {F3sqr == F3cda}\n")

F3cda66 = F3.make_cayley_dickson_algebra(version=1)
print(f"{F3cda66.description}")
print(f"F3sqr == F3cda66 ?: {F3sqr == F3cda66}\n")

F3cda53 = F3.make_cayley_dickson_algebra(version=2)
print(f"{F3cda53.description}")
print(f"F3sqr == F3cda53 ?: {F3sqr == F3cda53}\n")

Cayley-Dickson algebra based on F3, where mu = None, basic version.
F3sqr == F3cda ?: True

Cayley-Dickson algebra based on F3, where mu = 2, Schafer 1966 version.
F3sqr == F3cda66 ?: True

Cayley-Dickson algebra based on F3, where mu = 2, Schafer 1953 version.
F3sqr == F3cda53 ?: True



In [6]:
R4cda = R4.make_cayley_dickson_algebra()
print(f"{R4cda.description}")
print(f"R4sqr == R4cda ?: {R4sqr == R4cda}\n")

R4cda66 = R4.make_cayley_dickson_algebra(version=1)
print(f"{R4cda66.description}")
print(f"R4sqr == R4cda66 ?: {R4sqr == R4cda66}\n")

R4cda53 = R4.make_cayley_dickson_algebra(version=2)
print(f"{R4cda53.description}")
print(f"R4sqr == R4cda53 ?: {R4sqr == R4cda53}\n")

Cayley-Dickson algebra based on R4, where mu = None, basic version.
R4sqr == R4cda ?: True

Cayley-Dickson algebra based on R4, where mu = 3, Schafer 1966 version.
R4sqr == R4cda66 ?: True

Cayley-Dickson algebra based on R4, where mu = 3, Schafer 1953 version.
R4sqr == R4cda53 ?: True



In [7]:
F5cda = F5.make_cayley_dickson_algebra()
print(f"{F5cda.description}")
print(f"F5sqr == F5cda ?: {F5sqr == F5cda}\n")

F5cda66 = F5.make_cayley_dickson_algebra(version=1)
print(f"{F5cda66.description}")
print(f"F5sqr == F5cda66 ?: {F5sqr == F5cda66}\n")

F5cda53 = F5.make_cayley_dickson_algebra(version=2)
print(f"{F5cda53.description}")
print(f"F5sqr == F5cda53 ?: {F5sqr == F3cda53}\n")

Cayley-Dickson algebra based on F5, where mu = None, basic version.
F5sqr == F5cda ?: True

Cayley-Dickson algebra based on F5, where mu = 4, Schafer 1966 version.
F5sqr == F5cda66 ?: True

Cayley-Dickson algebra based on F5, where mu = 4, Schafer 1953 version.
F5sqr == F5cda53 ?: False



In [8]:
print(f"F5cda66 == F5cda53 ?: {F5sqr == F3cda53}\n")

F5cda66 == F5cda53 ?: False



In [9]:
F7cda = F7.make_cayley_dickson_algebra()
print(f"{F7cda.description}")
print(f"F7sqr == F7cda ?: {F7sqr == F7cda}\n")

F7cda66 = F7.make_cayley_dickson_algebra(version=1)
print(f"{F7cda66.description}")
print(f"F7sqr == F7cda66 ?: {F7sqr == F7cda66}\n")

F7cda53 = F7.make_cayley_dickson_algebra(version=2)
print(f"{F7cda53.description}")
print(f"F7sqr == F7cda53 ?: {F7sqr == F7cda53}\n")

Cayley-Dickson algebra based on F7, where mu = None, basic version.
F7sqr == F7cda ?: True

Cayley-Dickson algebra based on F7, where mu = 6, Schafer 1966 version.
F7sqr == F7cda66 ?: True

Cayley-Dickson algebra based on F7, where mu = 6, Schafer 1953 version.
F7sqr == F7cda53 ?: True



### Two Applications of Cayley-Dickson

In [10]:
%%time

F3quad = F3sqr.make_cayley_dickson_algebra()
#print(F3quad)

CPU times: user 725 ms, sys: 1.17 ms, total: 726 ms
Wall time: 726 ms


In [11]:
%%time

F3cda2 = F3cda.make_cayley_dickson_algebra()

F3cda2_66 = F3cda.make_cayley_dickson_algebra(version=1)

F3cda2_53 = F3cda.make_cayley_dickson_algebra(version=2)

CPU times: user 2.18 s, sys: 1.51 ms, total: 2.18 s
Wall time: 2.19 s


In [12]:
%%time

R4cda2 = R4cda.make_cayley_dickson_algebra()

CPU times: user 22.2 s, sys: 3.86 ms, total: 22.2 s
Wall time: 22.2 s


In [13]:
R4cda2.about(show_elements=False, show_tables=False)


** Ring **
Name: R4_Sqr_Sqr
Instance ID: 4421893648
Description: Cayley-Dickson algebra based on R4_Sqr, where mu = None, basic version.
Order: 256
Identity: '0:0:0:0'
Commutative? Yes
Cyclic?: No
Mult. Identity: '1:0:0:0'
Mult. Commutative? Yes
Zero Divisors: ['0:0:0:2', '0:0:1:1', '0:0:1:3', '0:0:2:0', '0:0:2:2', '0:0:3:1', '0:0:3:3', '0:1:0:1', '0:1:0:3', '0:1:1:0', '0:1:1:2', '0:1:2:1', '0:1:2:3', '0:1:3:0', '0:1:3:2', '0:2:0:0', '0:2:0:2', '0:2:1:1', '0:2:1:3', '0:2:2:0', '0:2:2:2', '0:2:3:1', '0:2:3:3', '0:3:0:1', '0:3:0:3', '0:3:1:0', '0:3:1:2', '0:3:2:1', '0:3:2:3', '0:3:3:0', '0:3:3:2', '1:0:0:1', '1:0:0:3', '1:0:1:0', '1:0:1:2', '1:0:2:1', '1:0:2:3', '1:0:3:0', '1:0:3:2', '1:1:0:0', '1:1:0:2', '1:1:1:1', '1:1:1:3', '1:1:2:0', '1:1:2:2', '1:1:3:1', '1:1:3:3', '1:2:0:1', '1:2:0:3', '1:2:1:0', '1:2:1:2', '1:2:2:1', '1:2:2:3', '1:2:3:0', '1:2:3:2', '1:3:0:0', '1:3:0:2', '1:3:1:1', '1:3:1:3', '1:3:2:0', '1:3:2:2', '1:3:3:1', '1:3:3:3', '2:0:0:0', '2:0:0:2', '2:0:1:1', '2:0:1:3', 

In [14]:
%%time

R4cda2_66 = R4cda.make_cayley_dickson_algebra(version=1)

CPU times: user 22.2 s, sys: 3.42 ms, total: 22.2 s
Wall time: 22.3 s


In [15]:
R4cda2 == R4cda2_66

False

In [16]:
R4cda2_66.about(show_elements=False, show_tables=False)


** Ring **
Name: R4_Sqr_CDA66
Instance ID: 5175850256
Description: Cayley-Dickson algebra based on R4_Sqr, where mu = 3:0, Schafer 1966 version.
Order: 256
Identity: '0:0:0:0'
Commutative? Yes
Cyclic?: No
Mult. Identity: '1:0:0:0'
Mult. Commutative? No
Zero Divisors: ['0:0:0:2', '0:0:1:1', '0:0:1:3', '0:0:2:0', '0:0:2:2', '0:0:3:1', '0:0:3:3', '0:1:0:1', '0:1:0:3', '0:1:1:0', '0:1:1:2', '0:1:2:1', '0:1:2:3', '0:1:3:0', '0:1:3:2', '0:2:0:0', '0:2:0:2', '0:2:1:1', '0:2:1:3', '0:2:2:0', '0:2:2:2', '0:2:3:1', '0:2:3:3', '0:3:0:1', '0:3:0:3', '0:3:1:0', '0:3:1:2', '0:3:2:1', '0:3:2:3', '0:3:3:0', '0:3:3:2', '1:0:0:1', '1:0:0:3', '1:0:1:0', '1:0:1:2', '1:0:2:1', '1:0:2:3', '1:0:3:0', '1:0:3:2', '1:1:0:0', '1:1:0:2', '1:1:1:1', '1:1:1:3', '1:1:2:0', '1:1:2:2', '1:1:3:1', '1:1:3:3', '1:2:0:1', '1:2:0:3', '1:2:1:0', '1:2:1:2', '1:2:2:1', '1:2:2:3', '1:2:3:0', '1:2:3:2', '1:3:0:0', '1:3:0:2', '1:3:1:1', '1:3:1:3', '1:3:2:0', '1:3:2:2', '1:3:3:1', '1:3:3:3', '2:0:0:0', '2:0:0:2', '2:0:1:1', '2:0

In [17]:
print(F3cda2)
print(f"F3cda2 == F3quad ?: {F3cda2 == F3quad} (This should be True)")
print(f"F3cda2 == F3cda2_66 ?: {F3cda2 == F3cda2_66}")
print(f"F3cda2 == F3cda2_53 ?: {F3cda2 == F3cda2_53}")
print(f"F3cda2_66 == F3cda2_53 ?: {F3cda2_66 == F3cda2_53}")

<Ring:F3_Sqr_Sqr, ID:4377202768>
F3cda2 == F3quad ?: True (This should be True)
F3cda2 == F3cda2_66 ?: False
F3cda2 == F3cda2_53 ?: False
F3cda2_66 == F3cda2_53 ?: False


In [18]:
print(len(F3quad.zero_divisors()))
print(len(F3cda2.zero_divisors()))
print(len(F3cda2_66.zero_divisors()))
print(len(F3cda2_53.zero_divisors()))

16
16
32
32


In [19]:
alg1 = F3cda2_66
alg2 = F3cda2_53

total = 0
count = 0
for a in alg1:
    for b in alg1:
        total += 1
        if not alg1.mult(a, b) == alg2.mult(a, b):
            count += 1
print(f"{count} unequal out of {total}")

5616 unequal out of 6561


In [20]:
alg1 = F3cda2_66
alg2 = F3cda2

total = 0
count = 0
for a in alg1:
    for b in alg1:
        total += 1
        if not alg1.mult(a, b) == alg2.mult(a, b):
            count += 1
print(f"{count} unequal out of {total}")

5184 unequal out of 6561


In [21]:
alg1 = F3cda2
alg2 = F3cda2_53

total = 0
count = 0
for a in alg1:
    for b in alg1:
        total += 1
        if not alg1.mult(a, b) == alg2.mult(a, b):
            count += 1
print(f"{count} unequal out of {total}")

5184 unequal out of 6561


## Norm Squared

This tests that the following relationship holds true: $\forall x, y \in F, N(x) \cdot N(y) = N(xy)$

In [22]:
A = F3cda2
ok = 0
not_ok = 0
for x in A:
    for y in A:
        if not (A.mult(A.norm(x), A.norm(y)) == A.norm(A.mult(x, y))):
            not_ok += 1
            All_OK = False
            # print(x, y)
        else:
            ok += 1

print(f"{A.name}: {ok} OK, {not_ok} not OK")

F3_Sqr_Sqr: 1073 OK, 5488 not OK


In [23]:
algebras = [F3, F3sqr, F3quad, F3cda, F3cda66, F3cda53,
            R4cda, R4cda66, R4cda53, R4cda2, F3cda2,
            R4cda2_66, F3cda2_66, F3cda2_53]

for A in algebras:
    ok = 0
    not_ok = 0
    for x in A:
        for y in A:
            if not (A.mult(A.norm(x), A.norm(y)) == A.norm(A.mult(x, y))):
                not_ok += 1
                All_OK = False
                # print(x, y)
            else:
                ok += 1
    print(f"{A.name}: {ok} OK, {not_ok} not OK")

F3: 9 OK, 0 not OK
F3_Sqr: 81 OK, 0 not OK
F3_Sqr_Sqr: 1073 OK, 5488 not OK
F3_Sqr: 81 OK, 0 not OK
F3_CDA66: 81 OK, 0 not OK
F3_CDA53: 81 OK, 0 not OK
R4_Sqr: 256 OK, 0 not OK
R4_CDA66: 256 OK, 0 not OK
R4_CDA53: 256 OK, 0 not OK
R4_Sqr_Sqr: 36352 OK, 29184 not OK
F3_Sqr_Sqr: 1073 OK, 5488 not OK
R4_Sqr_CDA66: 65536 OK, 0 not OK
F3_Sqr_CDA66: 6561 OK, 0 not OK
F3_Sqr_CDA53: 6561 OK, 0 not OK


## Check Conjugate

In [24]:
base = F3sqr
base2= F3quad
# base = R4cda
# base2 = R4cda2

elem = '1:2:2:1'
a, b = base2.split_element(elem)
print(f"  Element = {elem}")
conj_a = base.conj(a)
neg_b = base.inv(b)
print(f"conj({a}) = {conj_a}")
print(f"   -({b}) = {neg_b}")
conj_elem = ":".join([conj_a, neg_b])
print(f"conj({elem}) = {conj_elem}")

print(f"\nActual = {F3quad.conj(elem)}")

  Element = 1:2:2:1
conj(1:2) = 1:1
   -(2:1) = 1:2
conj(1:2:2:1) = 1:1:1:2

Actual = 1:1:1:2


In [25]:
# base = F3sqr
# base2= F3quad
base = R4cda
base2 = R4cda2

elem = '1:2:2:1'
a, b = base2.split_element(elem)
print(f"  Element = {elem}")
conj_a = base.conj(a)
neg_b = base.inv(b)
print(f"conj({a}) = {conj_a}")
print(f"   -({b}) = {neg_b}")
conj_elem = ":".join([conj_a, neg_b])
print(f"conj({elem}) = {conj_elem}")

print(f"\nActual = {F3quad.conj(elem)}")

  Element = 1:2:2:1
conj(1:2) = 1:2
   -(2:1) = 2:3
conj(1:2:2:1) = 1:2:2:3

Actual = 1:1:1:2


## Test to_dict

In [26]:
F3

Field(
'F3',
'Autogenerated Field of integers mod 3',
['0', '1', '2'],
[[0, 1, 2], [1, 2, 0], [2, 0, 1]],
[[0, 0, 0], [0, 1, 2], [0, 2, 1]]
)

In [27]:
F3.to_dict()

{'name': 'F3',
 'description': 'Autogenerated Field of integers mod 3',
 'elements': ['0', '1', '2'],
 'table': [[0, 1, 2], [1, 2, 0], [2, 0, 1]],
 'table2': [[0, 0, 0], [0, 1, 2], [0, 2, 1]]}

In [28]:
F3cda

Field(
'F3_Sqr',
'Cayley-Dickson algebra based on F3, where mu = None, basic version.',
['0:0', '0:1', '0:2', '1:0', '1:1', '1:2', '2:0', '2:1', '2:2'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 0, 4, 5, 3, 7, 8, 6], [2, 0, 1, 5, 3, 4, 8, 6, 7], [3, 4, 5, 6, 7, 8, 0, 1, 2], [4, 5, 3, 7, 8, 6, 1, 2, 0], [5, 3, 4, 8, 6, 7, 2, 0, 1], [6, 7, 8, 0, 1, 2, 3, 4, 5], [7, 8, 6, 1, 2, 0, 4, 5, 3], [8, 6, 7, 2, 0, 1, 5, 3, 4]],
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 3, 1, 7, 4, 2, 8, 5], [0, 3, 6, 2, 5, 8, 1, 4, 7], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 7, 5, 4, 2, 6, 8, 3, 1], [0, 4, 8, 5, 6, 1, 7, 2, 3], [0, 2, 1, 6, 8, 7, 3, 5, 4], [0, 8, 4, 7, 3, 2, 5, 1, 6], [0, 5, 7, 8, 1, 3, 4, 6, 2]],
{'0:0': '0:0', '0:1': '0:2', '0:2': '0:1', '1:0': '1:0', '1:1': '1:2', '1:2': '1:1', '2:0': '2:0', '2:1': '2:2', '2:2': '2:1'}
)

In [29]:
F3cda_dict = F3cda.to_dict()
F3cda_dict

{'name': 'F3_Sqr',
 'description': 'Cayley-Dickson algebra based on F3, where mu = None, basic version.',
 'elements': ['0:0', '0:1', '0:2', '1:0', '1:1', '1:2', '2:0', '2:1', '2:2'],
 'table': [[0, 1, 2, 3, 4, 5, 6, 7, 8],
  [1, 2, 0, 4, 5, 3, 7, 8, 6],
  [2, 0, 1, 5, 3, 4, 8, 6, 7],
  [3, 4, 5, 6, 7, 8, 0, 1, 2],
  [4, 5, 3, 7, 8, 6, 1, 2, 0],
  [5, 3, 4, 8, 6, 7, 2, 0, 1],
  [6, 7, 8, 0, 1, 2, 3, 4, 5],
  [7, 8, 6, 1, 2, 0, 4, 5, 3],
  [8, 6, 7, 2, 0, 1, 5, 3, 4]],
 'table2': [[0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 6, 3, 1, 7, 4, 2, 8, 5],
  [0, 3, 6, 2, 5, 8, 1, 4, 7],
  [0, 1, 2, 3, 4, 5, 6, 7, 8],
  [0, 7, 5, 4, 2, 6, 8, 3, 1],
  [0, 4, 8, 5, 6, 1, 7, 2, 3],
  [0, 2, 1, 6, 8, 7, 3, 5, 4],
  [0, 8, 4, 7, 3, 2, 5, 1, 6],
  [0, 5, 7, 8, 1, 3, 4, 6, 2]],
 'conj_map': {'0:0': '0:0',
  '0:1': '0:2',
  '0:2': '0:1',
  '1:0': '1:0',
  '1:1': '1:2',
  '1:2': '1:1',
  '2:0': '2:0',
  '2:1': '2:2',
  '2:2': '2:1'}}

In [31]:
F3cda_from_dict = alg.make_finite_algebra(F3cda_dict)
F3cda_from_dict

Field(
'F3_Sqr',
'Cayley-Dickson algebra based on F3, where mu = None, basic version.',
['0:0', '0:1', '0:2', '1:0', '1:1', '1:2', '2:0', '2:1', '2:2'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 0, 4, 5, 3, 7, 8, 6], [2, 0, 1, 5, 3, 4, 8, 6, 7], [3, 4, 5, 6, 7, 8, 0, 1, 2], [4, 5, 3, 7, 8, 6, 1, 2, 0], [5, 3, 4, 8, 6, 7, 2, 0, 1], [6, 7, 8, 0, 1, 2, 3, 4, 5], [7, 8, 6, 1, 2, 0, 4, 5, 3], [8, 6, 7, 2, 0, 1, 5, 3, 4]],
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 6, 3, 1, 7, 4, 2, 8, 5], [0, 3, 6, 2, 5, 8, 1, 4, 7], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 7, 5, 4, 2, 6, 8, 3, 1], [0, 4, 8, 5, 6, 1, 7, 2, 3], [0, 2, 1, 6, 8, 7, 3, 5, 4], [0, 8, 4, 7, 3, 2, 5, 1, 6], [0, 5, 7, 8, 1, 3, 4, 6, 2]],
{'0:0': '0:0', '0:1': '0:2', '0:2': '0:1', '1:0': '1:0', '1:1': '1:2', '1:2': '1:1', '2:0': '2:0', '2:1': '2:2', '2:2': '2:1'}
)

In [32]:
R4.to_dict()

{'name': 'R4',
 'description': 'Autogenerated Ring of integers mod 4',
 'elements': ['0', '1', '2', '3'],
 'table': [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]],
 'table2': [[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 0, 2], [0, 3, 2, 1]]}

In [34]:
R4cda_dict = R4cda.to_dict()
R4cda_dict

{'name': 'R4_Sqr',
 'description': 'Cayley-Dickson algebra based on R4, where mu = None, basic version.',
 'elements': ['0:0',
  '0:1',
  '0:2',
  '0:3',
  '1:0',
  '1:1',
  '1:2',
  '1:3',
  '2:0',
  '2:1',
  '2:2',
  '2:3',
  '3:0',
  '3:1',
  '3:2',
  '3:3'],
 'table': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
  [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12],
  [2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13],
  [3, 0, 1, 2, 7, 4, 5, 6, 11, 8, 9, 10, 15, 12, 13, 14],
  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3],
  [5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12, 1, 2, 3, 0],
  [6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 2, 3, 0, 1],
  [7, 4, 5, 6, 11, 8, 9, 10, 15, 12, 13, 14, 3, 0, 1, 2],
  [8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7],
  [9, 10, 11, 8, 13, 14, 15, 12, 1, 2, 3, 0, 5, 6, 7, 4],
  [10, 11, 8, 9, 14, 15, 12, 13, 2, 3, 0, 1, 6, 7, 4, 5],
  [11, 8, 9, 10, 15, 12, 13, 14, 3, 0, 1, 2, 7, 4, 5, 6],
  [12, 13, 14, 15, 0, 1, 2, 3, 4

In [35]:
R4cda_from_dict = alg.make_finite_algebra(R4cda_dict)
R4cda_from_dict

Ring(
'R4_Sqr',
'Cayley-Dickson algebra based on R4, where mu = None, basic version.',
['0:0', '0:1', '0:2', '0:3', '1:0', '1:1', '1:2', '1:3', '2:0', '2:1', '2:2', '2:3', '3:0', '3:1', '3:2', '3:3'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12], [2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13], [3, 0, 1, 2, 7, 4, 5, 6, 11, 8, 9, 10, 15, 12, 13, 14], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3], [5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12, 1, 2, 3, 0], [6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 2, 3, 0, 1], [7, 4, 5, 6, 11, 8, 9, 10, 15, 12, 13, 14, 3, 0, 1, 2], [8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7], [9, 10, 11, 8, 13, 14, 15, 12, 1, 2, 3, 0, 5, 6, 7, 4], [10, 11, 8, 9, 14, 15, 12, 13, 2, 3, 0, 1, 6, 7, 4, 5], [11, 8, 9, 10, 15, 12, 13, 14, 3, 0, 1, 2, 7, 4, 5, 6], [12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [13, 14, 15, 12, 1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8], [14, 15, 12, 1

In [37]:
R4cda_from_dict.conj('2:3')

'2:1'